In the new architecture of the DiffGFDN, we have decoupled FDNs, each having the common decay times associated with each room in the coupled space, The block diagonal mixing matrix, $\mathbf{A}$ and the input and output gains, $\mathbf{b,c}$ are determined using colourless FDN optimisation. The only thing left to be determined is the source and receiver filters, $\mathbf{g_i}(z), \mathbf{g_o}(z)$. Using the common-slopes model, we see that these gains model the position-dependent amplitudes associated with each common slope. Can we find these filters by using a rank-1 decomposition of the common slope amplitudes in each frequency band?

Let's first start by investigating the broadband case, where the source-receiver filters are replaced with broadband gains. We test our hypothesis on a synthetically generated dataset of shaped white noise, which has two slopes, each with decay time $0.5$s and $1.7$s. The amplitudes are spatial functions of the room geometry and source-receiver positions. Let the amplitude of the $k$th slope at source position $\mathbf{x_s}$ and receiver position $\mathbf{x_r}$ be denoted as $A_k(\mathbf{x_s, x_r}).$ On doing a common-slope analysis of RIRs at source positions $\mathbf{x_{s_1}} \cdots, \mathbf{x_{s_M}}$ and receiver multiple receiver positions $\mathbf{x_{r_1}}, \cdots, \mathbf{x_{r_N}}$, we can get a matrix of amplitudes for the $k$th slope, 

\begin{aligned}
\mathbf{A}_k = \begin{bmatrix}A_k(\mathbf{x_{s_1}, x_{r_1}}) & A_k(\mathbf{x_{s_1}, x_{r_2}}) & \cdots  & A_k(\mathbf{x_{s_1}, x_{r_N}}) \\
\vdots & \vdots & \ddots & \vdots \\
A_k(\mathbf{x_{s_M}, x_{r_1}}) & A_k(\mathbf{x_{s_M}, x_{r_2}}) & \cdots  & A_k(\mathbf{x_{s_M}, x_{r_N}})
\end{bmatrix}
\end{aligned}

We can do a rank-1 decomposition of this matrix using singular value decomposition
\begin{aligned}
\mathbf{A}_k  &= \mathbf{U_k} \Sigma_k \mathbf{V_k}^{-1} \\
& \approx \sigma_{1k} \mathbf{u}_{1k} \mathbf{v}_{1k}^{H}
\end{aligned}
where $\mathbf{u}_{1k} \in \mathbb{C}^{M \times 1}, \mathbf{v}_{1k} \in \mathbb{C}^{N \times 1}$ are the left and right singular vectors associated with the largest singular value, $\sigma_{1k}$. Using this decomposition, we can set the source and receiver gains for the $k$th FDN as, 

\begin{aligned}
\mathbf{g_{i_k}} &= \begin{bmatrix} g_{i_k}(\mathbf{x_{s_1}}) & \cdots & g_{i_k}(\mathbf{x_{s_M}})\end{bmatrix}^T = \sqrt{\sigma_{1k}}\mathbf{u}_{1k} \\
\mathbf{g_{o_k}} &= \begin{bmatrix} g_{o_k}(\mathbf{x_{r_1}}) & \cdots & g_{o_k}(\mathbf{x_{r_N}})\end{bmatrix}^T = \sqrt{\sigma_{1k}}\mathbf{v}_{1k}
\end{aligned}

We can pre-compute these gains from common-slope analysis and interpolate between them for dynamic source and listener positions. Now, if we do not have a regular grid of source-receiver positions, for example, for source location $\mathbf{x_{s_m}}$, we only have measured RIRs at $P < N$ receiver positions, then some entries in the matrix $\mathbf{A}_k$ will be 0. We can still recover the source and receiver gains by minimising the loss:

\begin{aligned}
\mathcal{L}(\mathbf{g}_{ik}, \mathbf{g}_{ok}) := ||\mathbf{g}_{ik} \mathbf{g}_{ok}^T - \mathbf{A}_k||_F
\end{aligned}
We can initialise $\mathbf{g}_{ik}, \mathbf{g}_{ok}$ with the rank-1 decomposition suggested above, and then use gradient descent to converge to an optimal solution of $\mathbf{g}_{ik}^*, \mathbf{g}_{ok}^*$
To fix scaling ambiguities, i.e., the optimum value of the source gains is found as, $\mathbf{g}^*_{ik}\alpha$ and the optimum values of receiver gains is found as, $\frac{\mathbf{g}^*_{ok}}{\alpha}$ for some non-zero $\alpha$, a modified loss is considered,

\begin{aligned}
\mathcal{L}^* (\mathbf{g}_{ik}, \mathbf{g}_{ok}) := ||\mathbf{g}_{ik} \mathbf{g}_{ok}^T - \mathbf{A}_k||_F + \lambda (\mathbf{g}_{ik}^T \mathbf{g}_{ik} - \mathbf{g}_{ok}^T\mathbf{g}_{ok})
\end{aligned}

In [ ]:
import numpy as np
import torch
import re
import os
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.io import loadmat
from torch import nn
from numpy.typing import ArrayLike
from typing import Optional, List
from IPython import display
from tqdm import tqdm
import soundfile as sf
from loguru import logger
from copy import deepcopy
from scipy.io import savemat, loadmat
os.chdir('..')  # This changes the working directory to DiffGFDN

from diff_gfdn.dataloader import load_dataset, RIRData, RoomDataset
from diff_gfdn.config.config import DiffGFDNConfig, CouplingMatrixType
from diff_gfdn.utils import is_unitary, db2lin, db, ms_to_samps, get_response
from diff_gfdn.plot import (plot_edr, animate_coupled_feedback_matrix, plot_subband_edc, 
                            plot_learned_svf_response, plot_amps_in_space, plot_edc_error_in_space)
from diff_gfdn.solver import convert_common_slopes_rir_to_room_dataset
from diff_gfdn.analysis import get_decay_fit_net_params
from diff_gfdn.low_rank_decomp import run_low_rank_decomp
from src.run_model import load_and_validate_config

### Create config file and dataset

In [ ]:
config_path = 'data/config/'
fig_path = 'figures/'
config_name = 'synth_data_broadband_two_coupled_rooms_colorless_prototype_low_rank_decomp.yml'
config_file = config_path + config_name
config_dict = load_and_validate_config(config_file,
                                       DiffGFDNConfig)
room_data = convert_common_slopes_rir_to_room_dataset(config_dict.room_dataset_path, 
                                                      num_freq_bins=config_dict.trainer_config.num_freq_bins,
                                                     )
ir_dir = Path(config_dict.trainer_config.ir_dir).resolve()
if not os.path.exists(ir_dir):
    os.makedirs(ir_dir)                                                      
config_dict = config_dict.copy(update={"num_groups": room_data.num_rooms})

trainer_config = config_dict.trainer_config
# prepare the training and validation data for DiffGFDN
if trainer_config.batch_size != room_data.num_freq_bins:
    trainer_config = trainer_config.copy(
        update={"batch_size": room_data.num_freq_bins})

mixing_time_samp = ms_to_samps(20.0, config_dict.sample_rate)
crop_end_samp = ms_to_samps(5.0, config_dict.sample_rate)

### Run the DiffGFDN with the source-receiver gains

In [ ]:
# Define the folder path and the extension
folder_path = Path(trainer_config.train_dir).resolve()
file_extension = ".mat"  # Change to desired extension, e.g., ".pkl", ".csv"
file_found = False

# Check for files with the given extension
files_with_extension = [f for f in os.listdir(folder_path) if f.endswith(file_extension)]

if files_with_extension:
    file_found = True
    # If one or more files are found, process the first one (or loop over all)
    for file_name in files_with_extension:
        file_path = os.path.join(folder_path, file_name)
        logger.info(f"Found file: {file_name}, loading saved parameters")
        mdic = loadmat(file_path)
    all_rir_recons = mdic['approx_rir']
else:
    logger.info("Running the training for source-receiver grid")
    all_rir_recons = run_low_rank_decomp(config_dict)

### Evaluate the output at one position

In [ ]:
def get_edc_params(rir: ArrayLike, n_slopes: int, fs:float):
    est_params_decay_net, norm_vals, fitted_edc_subband = get_decay_fit_net_params(rir, None, n_slopes, fs)
    est_t60 = np.mean(est_params_decay_net[0], axis=0)
    est_amp = np.mean(est_params_decay_net[1], axis=0)
    est_noise = np.mean(est_params_decay_net[2], axis=0)
    fitted_edc = torch.squeeze(torch.mean(fitted_edc_subband, dim=0))
    return est_t60, est_amp, est_noise, fitted_edc

def plot_final_decay_fit_net_edc(num_groups:int, fs: float, og_amps: List, est_amps: List, 
                                 og_edc: ArrayLike, synth_edc:ArrayLike, 
                                 src_pos:Optional[List]=None, rec_pos: Optional[List]=None):
    
    time = np.linspace(0, (len(og_edc) - 1) / fs, len(og_edc))
    fig, ax = plt.subplots(figsize=(6, 4))

    ax.plot(time, db(og_edc, is_squared=True), linestyle='-', label=f'Original estimated EDC (norm)')
    ax.plot(time[:len(synth_edc)], db(synth_edc, is_squared=True), linestyle='--', label=f'Final estimated EDC (norm)')
    ax.plot(np.zeros(num_groups), db(og_amps, is_squared=True), 'kx', label='Original amps')
    ax.plot(np.zeros(num_groups), db(est_amps, is_squared=True), 'gd', label='Synth amps')

    if src_pos is not None and rec_pos is not None:
        ax.set_title(f"Source pos = {np.round(src_pos, 2)}, receiver pos = {np.round(rec_pos, 2)}")

    ax.legend()
    plt.show()

def plot_edc(h_true: ArrayLike, h_approx: ArrayLike, fs: float, mixing_time_ms:float=20.0, og_amps: Optional[List]=None):
    """Plot true and synthesised EDC curves"""
    
    mixing_time_samp = ms_to_samps(mixing_time_ms, fs)
    crop_end_samp = ms_to_samps(5.0, fs)
    
    trunc_true_ir = h_true[mixing_time_samp:-crop_end_samp]
    true_edf = np.flipud(np.cumsum(np.flipud(trunc_true_ir**2), axis=-1))
    time = np.linspace(0, (len(trunc_true_ir) - 1) / fs,
                       len(trunc_true_ir))
  
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(time, db(true_edf, is_squared=True), label='True EDF')
    if og_amps is not None:
        ax.plot(np.zeros(len(og_amps)), db(og_amps, is_squared=True), 'kd')
    trunc_approx_ir = h_approx[mixing_time_samp: mixing_time_samp + len(trunc_true_ir)]
    synth_edf = np.flipud(np.cumsum(np.flipud(trunc_approx_ir**2), axis=-1))
    ax.plot(time, db(synth_edf, is_squared=True), label='Synth EDF')
    ax.legend()
    fig.savefig(Path(f'{fig_path}/compare_synth_edf_{config_name}.png').resolve())
    plt.show()

In [ ]:
src_idx = np.random.randint(0, high=room_data.num_src, size=1, dtype=int)
rec_idx = np.random.randint(0, high=room_data.num_rec, size=1, dtype=int)
source_pos = np.squeeze(room_data.source_position[src_idx, :])
rec_pos = np.squeeze(room_data.receiver_position[rec_idx, :])

true_ir = np.squeeze(room_data.rirs[src_idx, rec_idx, :])
approx_ir = np.squeeze(all_rir_recons[src_idx, rec_idx, :])
amplitudes = np.squeeze(room_data.amplitudes[src_idx, rec_idx, :])

plt.plot(np.stack((true_ir, approx_ir), axis=-1))
plt.xlim([0, int(1.5 * config_dict.sample_rate)])
plt.show()

trunc_true_ir = true_ir[mixing_time_samp:-crop_end_samp]
trunc_approx_ir = approx_ir[mixing_time_samp:-crop_end_samp]
og_est_t60, og_est_amp, og_noise_floor, og_fitted_edc = get_edc_params(trunc_true_ir, config_dict.num_groups, config_dict.sample_rate)
est_t60, est_amp, _, fitted_edc = get_edc_params(trunc_approx_ir, config_dict.num_groups, config_dict.sample_rate)

plot_final_decay_fit_net_edc(config_dict.num_groups, config_dict.sample_rate, amplitudes, est_amp, og_fitted_edc, fitted_edc, 
                             src_pos=source_pos, rec_pos=rec_pos)
plot_edc(true_ir, approx_ir, config_dict.sample_rate, og_amps=amplitudes)

### Get the overall amplitude and EDC mismatch as a function of spatial location

In [ ]:
from importlib import reload
import diff_gfdn
reload(diff_gfdn.plot)
from diff_gfdn.plot import plot_amps_in_space, plot_edc_error_in_space

plot_edc_error_in_space(room_data, all_rir_recons, room_data.receiver_position, freq_to_plot=None, 
                         scatter=True, save_path=f'{fig_path}/{config_name}', pos_sorted=True)

estimated_amps = plot_amps_in_space(room_data, all_rir_recons, room_data.receiver_position, 
                                    freq_to_plot=None, scatter=True, save_path=f'{fig_path}/{config_name}', pos_sorted=True)

In [ ]:
if not file_found:
    mdic = {'src_pos' : room_data.source_position, 'rec_pos': room_data.receiver_position, 
            'approx_rir': all_rir_recons, 'est_amps': estimated_amps}
    save_path = Path(config_dict.trainer_config.train_dir + 'low_rank_decomp_GFDN_approx_rirs.mat').resolve()
    savemat(save_path, mdic)